# 載入 package

In [ ]:
%pip install -q --upgrade transformers
%pip install -q --upgrade datasets
%pip install -q --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize

import re
import numpy as np
from tqdm import tqdm
import time
import torch
import math
import pandas as pd

from torch import cuda
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import Dataset as hf_Dataset
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader
from transformers import default_data_collator
from torch.optim import AdamW
from accelerate import Accelerator
from transformers import get_scheduler
from tqdm.notebook import tqdm as tqdm_notebook

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# 載入模型

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', add_special_tokens=True)
model = AutoModelForMaskedLM.from_pretrained('distilbert/distilbert-base-uncased')
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.

In [ ]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

66985530


In [ ]:
tokenizer.is_fast

True

# 載入資料

## 2022/11 train

In [ ]:
df_11_train = pd.read_pickle("/content/ap_table_202211_abnormal_train.pkl")

## 2022/11 - evaluation data

In [ ]:
df_val = pd.read_pickle("/content/ap_table_202211_abnormal_val.pkl")

## 2022/11

In [ ]:
df_11_test = pd.read_pickle("/content/ap_table_202211_abnormal_test.pkl")

## 2022/12

In [ ]:
df_12_test = pd.read_pickle("/content/ap_table_202212_abnormal_test.pkl")

## 2023/01

In [ ]:
df_01_test = pd.read_pickle("/content/ap_table_202301_abnormal_test.pkl")

## 2023/02

In [ ]:
df_02_test = pd.read_pickle("/content/ap_table_202302_abnormal_test.pkl")

## 2023/03

In [ ]:
df_03_test = pd.read_pickle("/content/ap_table_202303_abnormal_test.pkl")

## 2023/04

In [ ]:
df_04_test = pd.read_pickle("/content/ap_table_202304_abnormal_test.pkl")

## 2023/05

In [ ]:
df_05_test = pd.read_pickle("/content/ap_table_202305_abnormal_test.pkl")

## 2023/06

In [ ]:
df_06_test = pd.read_pickle("/content/ap_table_202306_abnormal_test.pkl")

## 2023/07

In [ ]:
df_07_test = pd.read_pickle("/content/ap_table_202307_abnormal_test.pkl")

## 2023/08

In [ ]:
df_08_test = pd.read_pickle("/content/ap_table_202308_abnormal_test.pkl")

## 2023/09

In [ ]:
df_09_test = pd.read_pickle("/content/ap_table_202309_abnormal_test.pkl")

## 2023/10

In [ ]:
df_10_test = pd.read_pickle("/content/ap_table_202310_abnormal_test.pkl")

# Reset Index

In [ ]:
df_11_train = df_11_train.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_val = df_val.reset_index(drop = True)

In [ ]:
df_11_test = df_11_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_12_test = df_12_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_01_test = df_01_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_02_test = df_02_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_03_test = df_03_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_04_test = df_04_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_05_test = df_05_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_06_test = df_06_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_07_test = df_07_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_08_test = df_08_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_09_test = df_09_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_10_test = df_10_test.reset_index(drop=True).sample(frac=1).reset_index(drop=True)

# Segmentaion Funtion

In [ ]:
def tokenization_addspace_new(s):
  word_list = wordpunct_tokenize(s.replace('[SEP]', 'Æ ')) # [SEP] 用 Æ 取代
  new_string = ""
  for i in range(len(word_list)):
    temp = re.split('([^a-zA-Z0-9])',word_list[i]) # 用特殊字元做文字分割
    for j in range(len(temp)):
      if temp[j] != "":  # 如果不是空字元
        if j != len(temp) - 2 and j != len(temp) - 1: # 如果不是最後兩個字元，則加空格
          if temp[j] == 'Æ':
            new_string += '[SEP]' + " " # 把 Æ 換回成 [SEP]
          else:
            new_string += temp[j] + " "
        else:
          if temp[j] == 'Æ':
            new_string += '[SEP]'
          else:
            new_string += temp[j]
    if i != len(word_list) - 1:
      new_string += " "  # 加空格
  return new_string.lower()  # 全部變小寫

# 文字前處理

In [ ]:
df_11_train['text'] = ""
df_11_train['text'] = df_11_train['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_val['text'] = ""
df_val['text'] = df_val['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

In [ ]:
df_11_test['text'] = ""
df_11_test['text'] = df_11_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_12_test['text'] = ""
df_12_test['text'] = df_12_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_01_test['text'] = ""
df_01_test['text'] = df_01_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_02_test['text'] = ""
df_02_test['text'] = df_02_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_03_test['text'] = ""
df_03_test['text'] = df_03_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_04_test['text'] = ""
df_04_test['text'] = df_04_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_05_test['text'] = ""
df_05_test['text'] = df_05_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_06_test['text'] = ""
df_06_test['text'] = df_06_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_07_test['text'] = ""
df_07_test['text'] = df_07_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_08_test['text'] = ""
df_08_test['text'] = df_08_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_09_test['text'] = ""
df_09_test['text'] = df_09_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

df_10_test['text'] = ""
df_10_test['text'] = df_10_test['clean_payload_list'].apply(lambda x: tokenization_addspace_new(x))

In [ ]:
df_11_train = df_11_train[df_11_train.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_val = df_val[df_val.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)

In [ ]:
df_11_test = df_11_test[df_11_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_12_test = df_12_test[df_12_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_01_test = df_01_test[df_01_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_02_test = df_02_test[df_02_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_03_test = df_03_test[df_03_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_04_test = df_04_test[df_04_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_05_test = df_05_test[df_05_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_06_test = df_06_test[df_06_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_07_test = df_07_test[df_07_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_08_test = df_08_test[df_08_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_09_test = df_09_test[df_09_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)
df_10_test = df_10_test[df_10_test.text != ""].reset_index(drop=True).sample(frac=1).reset_index(drop=True)

In [ ]:
print("2022/11 train: ", len(df_11_train))

2022/11 train:  140648


In [ ]:
df_11_train.Tactic_concate.value_counts()

Tactic_concate
Reconnaissance (TA0043)          52703
Initial Access (TA0001)          31654
Command and Control (TA0011)     28241
Credential Access (TA0006)        8967
Collection (TA0009)               8155
Execution (TA0002)                7920
Exfiltration (TA0010)             1394
Persistence (TA0003)               807
Resource Development (TA0042)      458
Privilege Escalation (TA0004)      182
Discovery (TA0007)                 113
Lateral Movement (TA0008)           54
Name: count, dtype: int64

# 建立 dataset - train

In [ ]:
new_df = df_11_train[["text"]]

In [ ]:
dataset = hf_Dataset.from_pandas(new_df)

In [ ]:
def tokenize_function(examples):
  result = tokenizer(examples["text"])
  if tokenizer.is_fast:
      result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
  return result

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Map:   0%|          | 0/140648 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 140648
})

In [ ]:
chunk_size = 128

In [ ]:
def group_texts(examples):

    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}  # 合併所有文字
    total_length = len(concatenated_examples[list(examples.keys())[0]])  # 計算合併後的長度
    total_length = (total_length // chunk_size) * chunk_size  # 如果 chunk 小於 chunk_size 則捨棄
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]  # 用 chunk_size 來切分文字
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/140648 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 177033
})

In [ ]:
tokenizer.decode(lm_datasets[1]["input_ids"])

'649 [ sep ] connection : close [ sep ] p3p : cp = " cao psa our " [ sep ] expires : thu, 01 jan 1970 00 : 00 : 00 gmt [ sep ] cache - control : no - store, no - cache, must - revalidate, post - check = 0, pre - check = 0 [ sep ] pragma : no - cache [ sep ] < html > [ sep ] < head > [ sep ] < title > this page is not available < / title > [ sep ] < meta http - equiv = " content - type "'

In [ ]:
tokenizer.decode(lm_datasets[1]["labels"])

'649 [ sep ] connection : close [ sep ] p3p : cp = " cao psa our " [ sep ] expires : thu, 01 jan 1970 00 : 00 : 00 gmt [ sep ] cache - control : no - store, no - cache, must - revalidate, post - check = 0, pre - check = 0 [ sep ] pragma : no - cache [ sep ] < html > [ sep ] < head > [ sep ] < title > this page is not available < / title > [ sep ] < meta http - equiv = " content - type "'

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] get / [MASK]search / sqlite [MASK] main. php http [MASK] version [ sep ] host : ip [ sep ] accept - language : en - us [MASK] q = 0. 6 [MASK] en ; q = 0. 4 [ sep ] referer : http : / / ip / [ sep [MASK] user - agent : mozilla / [MASK] ( [MASK] nt version ; rv : 9 [MASK] [MASK]. 1 ) activistscko / 20100101 firefox / version [SEP] [CLS] http / version 503 service unavailable [ sep ] content - type : text / html ; charset = [MASK]f - [MASK] [ sep ] content - length :'

'>>> 649 [MASK] sep ] [MASK] : [MASK] [ sep ] p3 [MASK] [MASK] cp = " cao psa our " [ [MASK] ] expires : th albany, [MASK] jan [MASK] [MASK] : 00 : 00 gmt [ sep ] cache - control : [MASK] - [MASK], [MASK] - cache, must - revalidate, post - [MASK] = 0, pre [MASK] check = 0 [ sep [MASK] pragma : no - cache [ sep ] < html > [ sep ] < head > [ sep ] < [MASK] > this page [MASK] [MASK] available < / title > [ sep ] < meta http - [MASK]quibahn = " content - type "'


In [ ]:
lm_datasets = lm_datasets.remove_columns(["word_ids"])

# 建立 dataset - val

In [ ]:
new_val_df = df_val[["text"]]

In [ ]:
dataset_val = hf_Dataset.from_pandas(new_val_df)

In [ ]:
tokenized_datasets_val = dataset_val.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets_val

Map:   0%|          | 0/17582 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids'],
    num_rows: 17582
})

In [ ]:
lm_datasets_val = tokenized_datasets_val.map(group_texts, batched=True)
lm_datasets_val

Map:   0%|          | 0/17582 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 21861
})

In [ ]:
lm_datasets_val = lm_datasets_val.remove_columns(["word_ids"])

In [ ]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)  # 加入隨機 mask
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}  # 為原本的 column 都再建立新的 masked column

In [ ]:
val_dataset = lm_datasets_val.map(
    insert_random_mask,
    batched=True,
    remove_columns=lm_datasets_val.column_names,
)
val_dataset = val_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

Map:   0%|          | 0/21861 [00:00<?, ? examples/s]

# Train Model

In [ ]:
batch_size = 32
train_dataloader = DataLoader(
    lm_datasets,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

val_dataloader = DataLoader(
    lm_datasets_val, batch_size=batch_size, collate_fn=default_data_collator
)

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-4)

In [ ]:
num_train_epochs = 4
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0.1*num_training_steps,
    num_training_steps=num_training_steps,
)

In [ ]:
accelerator = Accelerator()
model, optimizer, lr_scheduler, train_dataloader, val_dataloader = accelerator.prepare(
    model, optimizer, lr_scheduler, train_dataloader, val_dataloader
)

In [ ]:
progress_bar = tqdm(range(num_training_steps))
start_time = time.time()

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()  # Evaluation
    losses = []
    for step, batch in enumerate(val_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

    losses = torch.cat(losses)
    losses = losses[: len(val_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))  # 計算 perplexity
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

print("\n--- %s seconds ---" % (time.time() - start_time))

 25%|██▌       | 5535/22132 [12:24<30:15:32,  6.56s/it]

>>> Epoch 0: Perplexity: 1.2224093446045428


 50%|█████     | 11068/22132 [24:48<20:09:23,  6.56s/it]

>>> Epoch 1: Perplexity: 1.2155055442097185


 75%|███████▌  | 16601/22132 [37:13<10:04:01,  6.55s/it]

>>> Epoch 2: Perplexity: 1.1953500255603215


100%|██████████| 22132/22132 [49:06<00:00,  7.83it/s]

>>> Epoch 3: Perplexity: 1.2009661706860062

--- 2977.493255853653 seconds ---


# 儲存模型

In [ ]:
unwrapped_model = accelerator.unwrap_model(model)

In [ ]:
unwrapped_model.save_pretrained("distilbert_sess")

In [ ]:
from transformers import DistilBertModel
bert = DistilBertModel.from_pretrained('/content/distilbert_sess')
bert = bert.to(device)

In [ ]:
bert

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [ ]:
total_params = sum(p.numel() for p in unwrapped_model.parameters() if p.requires_grad)
print(total_params)

66985530


# 產生 embedding

In [ ]:
def get_embedding(s):  # 產生每個 session 的 embedding
  inputs = tokenizer(s, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
  outputs = bert(**inputs, output_hidden_states=True)
  emb = outputs.last_hidden_state[0,0,:].detach().cpu().numpy()
  return emb

In [ ]:
from tqdm.notebook import tqdm as tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
df_11_train["bert_embedding"] = np.empty((len(df_11_train), 0)).tolist()
df_11_train["bert_embedding"] = df_11_train['text'].progress_apply(lambda x: get_embedding(x))

  0%|          | 0/140648 [00:00<?, ?it/s]

In [ ]:
start_time = time.time()

df_11_test["bert_embedding"] = np.empty((len(df_11_test), 0)).tolist()
df_11_test["bert_embedding"] = df_11_test['text'].progress_apply(lambda x: get_embedding(x))

df_12_test["bert_embedding"] = np.empty((len(df_12_test), 0)).tolist()
df_12_test["bert_embedding"] = df_12_test['text'].progress_apply(lambda x: get_embedding(x))

df_01_test["bert_embedding"] = np.empty((len(df_01_test), 0)).tolist()
df_01_test["bert_embedding"] = df_01_test['text'].progress_apply(lambda x: get_embedding(x))

df_02_test["bert_embedding"] = np.empty((len(df_02_test), 0)).tolist()
df_02_test["bert_embedding"] = df_02_test['text'].progress_apply(lambda x: get_embedding(x))

df_03_test["bert_embedding"] = np.empty((len(df_03_test), 0)).tolist()
df_03_test["bert_embedding"] = df_03_test['text'].progress_apply(lambda x: get_embedding(x))

df_04_test["bert_embedding"] = np.empty((len(df_04_test), 0)).tolist()
df_04_test["bert_embedding"] = df_04_test['text'].progress_apply(lambda x: get_embedding(x))

df_05_test["bert_embedding"] = np.empty((len(df_05_test), 0)).tolist()
df_05_test["bert_embedding"] = df_05_test['text'].progress_apply(lambda x: get_embedding(x))

df_06_test["bert_embedding"] = np.empty((len(df_06_test), 0)).tolist()
df_06_test["bert_embedding"] = df_06_test['text'].progress_apply(lambda x: get_embedding(x))

df_07_test["bert_embedding"] = np.empty((len(df_07_test), 0)).tolist()
df_07_test["bert_embedding"] = df_07_test['text'].progress_apply(lambda x: get_embedding(x))

df_08_test["bert_embedding"] = np.empty((len(df_08_test), 0)).tolist()
df_08_test["bert_embedding"] = df_08_test['text'].progress_apply(lambda x: get_embedding(x))

df_09_test["bert_embedding"] = np.empty((len(df_09_test), 0)).tolist()
df_09_test["bert_embedding"] = df_09_test['text'].progress_apply(lambda x: get_embedding(x))

df_10_test["bert_embedding"] = np.empty((len(df_10_test), 0)).tolist()
df_10_test["bert_embedding"] = df_10_test['text'].progress_apply(lambda x: get_embedding(x))

print("\n--- %s seconds ---" % (time.time() - start_time))

  0%|          | 0/17581 [00:00<?, ?it/s]

  0%|          | 0/15626 [00:00<?, ?it/s]

  0%|          | 0/15731 [00:00<?, ?it/s]

  0%|          | 0/26245 [00:00<?, ?it/s]

  0%|          | 0/34099 [00:00<?, ?it/s]

...

  0%|          | 0/41265 [00:00<?, ?it/s]

  0%|          | 0/43423 [00:00<?, ?it/s]

  0%|          | 0/55023 [00:00<?, ?it/s]

  0%|          | 0/54019 [00:00<?, ?it/s]


--- 3163.437351703644 seconds ---


In [ ]:
df_10_test

,Session_ID,Src_ISP,Datetime,Protocol,city,country,isp,iso_code,domain,session_time,...,batch,pkt_num,dist,AP_name,new_observed,sess_num,pot,Tactic_concate,text,bert_embedding
0,20231027_中華電信_http_1698397631.213202000,中華電信,2023-10-27 17:07:11,http,Bedong,Malaysia,TM Net,MY,None,1.698398e+09,...,20231027_18,1,0.000000,20230607_03_http_02_9,False,1,glastopf,Reconnaissance (TA0043),get http : / / ip : port / phpmyadmin - 2 . 5 ...,"[0.13496631, -0.8197597, -0.11757963, 0.215185..."
1,20231025_台灣固網_http_1698226985.764611000,台灣固網,2023-10-25 17:43:05,http,None,Singapore,Huawei Clouds,SG,None,1.698227e+09,...,20231025_18,1,0.033296,20231025_18_http_02_7,True,1,glastopf,Execution (TA0002),get / login . action / www . google . com / am...,"[0.5599696, -0.0609452, 0.18001336, 0.17122912..."
2,20231025_台灣固網_http_1698205491.731156000,台灣固網,2023-10-25 11:44:51,http,None,Singapore,Huawei Clouds,SG,None,1.698205e+09,...,20231025_12,1,0.037586,20231025_12_http_02_6,True,1,glastopf,Reconnaissance (TA0043),get / comments / vsadmin / www . google . com ...,"[0.6378824, 0.05388186, 0.133232, 0.48320225, ..."
3,20231025_台灣固網_http_1698226592.956601000,台灣固網,2023-10-25 17:36:32,http,Boydton,United States,Microsoft bingbot,US,msn.com,1.698227e+09,...,20231025_18,1,0.040928,20231025_18_http_02_1,True,1,glastopf,Reconnaissance (TA0043),get / . br / perl / comments / perl / includes...,"[0.8647306, -0.22206196, 0.38328916, 0.4304777..."
4,20231017_台灣固網_http_1697492765.494633000,台灣固網,2023-10-17 05:46:05,http,None,Singapore,Huawei Clouds,SG,None,1.697493e+09,...,20231017_06,1,0.014751,20231017_06_http_02_3,True,1,glastopf,Reconnaissance (TA0043),get / perl / comments / perl / components / co...,"[0.59895396, 0.028341182, 0.14055091, -0.06070..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54014,20231007_中嘉寬頻_http_1696670423.689320000,中嘉寬頻,2023-10-07 17:20:23,http,Morainvilliers,France,ONLINE S.A.S.,FR,poneytelecom.eu,1.696670e+09,...,20231007_18,1,0.000387,20230615_01_http_11_41,False,1,glastopf,Reconnaissance (TA0043),get / comments / 1ufabet . com / site / zomzom...,"[1.1589876, -0.40109587, 0.2959381, 0.5379931,..."
54015,20231027_中華電信_http_1698356678.858244000,中華電信,2023-10-27 05:44:38,http,None,Germany,Limitless Mobile GmbH,DE,vistream.de,1.698357e+09,...,20231027_06,1,0.000000,20231016_06_http_02_71,False,1,amun,Reconnaissance (TA0043),get / geoserver http / version [sep] host : ip...,"[0.08505114, -0.3757598, 0.63756394, 0.7303152..."
54016,20231024_台灣固網_http_1698162499.521063000,台灣固網,2023-10-24 23:48:19,http,None,Singapore,Huawei Clouds,SG,None,1.698162e+09,...,20231025_00,1,0.024411,20231025_00_http_02_4,True,1,glastopf,Reconnaissance (TA0043),get / login . action / www . google . com / am...,"[0.54179555, 0.08590417, 0.1639003, 0.35878497..."
54017,20231017_台灣固網_http_1697512011.164442000,台灣固網,2023-10-17 11:06:51,http,Ashburn,United States,Amazon.com,US,amazonaws.com,1.697512e+09,...,20231017_12,1,0.014396,20231017_12_http_02_3,True,1,glastopf,Credential Access (TA0006),get / login _dl / assets / file : / etc / typo...,"[0.710726, -0.025064671, 0.34774706, 0.2203105..."


In [ ]:
# 將 embedding 每個維度都變成一個 column
df_temp_11_train = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_11_train['bert_embedding'].values.tolist()], index=df_11_train.index)

In [ ]:
df_temp_11 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_11_test['bert_embedding'].values.tolist()], index=df_11_test.index)
df_temp_12 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_12_test['bert_embedding'].values.tolist()], index=df_12_test.index)
df_temp_01 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_01_test['bert_embedding'].values.tolist()], index=df_01_test.index)
df_temp_02 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_02_test['bert_embedding'].values.tolist()], index=df_02_test.index)
df_temp_03 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_03_test['bert_embedding'].values.tolist()], index=df_03_test.index)
df_temp_04 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_04_test['bert_embedding'].values.tolist()], index=df_04_test.index)
df_temp_05 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_05_test['bert_embedding'].values.tolist()], index=df_05_test.index)
df_temp_06 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_06_test['bert_embedding'].values.tolist()], index=df_06_test.index)
df_temp_07 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_07_test['bert_embedding'].values.tolist()], index=df_07_test.index)
df_temp_08 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_08_test['bert_embedding'].values.tolist()], index=df_08_test.index)
df_temp_09 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_09_test['bert_embedding'].values.tolist()], index=df_09_test.index)
df_temp_10 = pd.DataFrame([{x: y for x, y in enumerate(item)} for item in df_10_test['bert_embedding'].values.tolist()], index=df_10_test.index)

In [ ]:
# 將所需的資料 column 合併成最終的 table
df_final_11_train = pd.concat([df_temp_11_train, df_11_train[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)

In [ ]:
df_final_11 = pd.concat([df_temp_11, df_11_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_12 = pd.concat([df_temp_12, df_12_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_01 = pd.concat([df_temp_01, df_01_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_02 = pd.concat([df_temp_02, df_02_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_03 = pd.concat([df_temp_03, df_03_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_04 = pd.concat([df_temp_04, df_04_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_05 = pd.concat([df_temp_05, df_05_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_06 = pd.concat([df_temp_06, df_06_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_07 = pd.concat([df_temp_07, df_07_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_08 = pd.concat([df_temp_08, df_08_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_09 = pd.concat([df_temp_09, df_09_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)
df_final_10 = pd.concat([df_temp_10, df_10_test[['Session_ID', 'Src_ISP', 'session_time','clean_payload_list', 'country', 'city', 'ip_src', 'domain', 'iso_code', 'session_duration', 'payload_list', 'Tactic_concate']]], axis=1)